In [1]:
import gzip

In [59]:
vepfile = '../../../bsa_by_rnaseq/06_vep/vep_output/HER19_variant_effects'
samplesfile = '../../../bsa_by_rnaseq/06_vep/samples.txt'
outfile = '../../../bsa_by_rnaseq/06_vep/filtered_vep_output.txt'

target_chrom = 'Chr29'
target_start = 96000000
target_end = 112000000

In [64]:
snpDict = {}

fi = open(vepfile)

for line in fi:
    if line[0] == '#':
        continue
        
    chrom = line.split('_')[0]
    loc = int(line.split('_')[1])
    ref = line.split('_')[2].split('/')[0]
    alt = line.split('/')[1].split('\t')[0]
    gene = line.split('\t')[3]
    effect  = line.split('\t')[6]
    aa = line.split('\t')[10]
    codon = line.split('\t')[11]
    impact = line.rstrip().split('IMPACT=')[1].split(';')[0]

    if chrom == target_chrom and loc > target_start and loc < target_end:
        #print(chrom,loc,ref,alt,gene,effect,aa,codon,impact)
        snpDict[(chrom, loc)] = (chrom,loc,ref,alt,gene,effect,aa,codon,impact)
    
fi.close()

In [65]:
len(snpDict)

46

In [66]:
sampleDict = {}

fi = open(samplesfile)

for line in fi:
    sample, vcfile = line.rstrip().split('\t')
    sampleDict[sample] = {}
    
    fi = gzip.open(vcfile, 'rt')

    for line in fi:
        #print(line)

        if line[0] == '#': 
            continue

        col = line.rstrip().split('\t')
        chrom = col[0]
        pos = int(col[1])
        ref = col[3]
        alt = col[4]

        #if chrom == target_chrom and pos > target_start and pos < target_end:
            #print(chrom,pos,ref,alt)

        genotype = col[9].split(':')[0]
        allele_depth = col[9].split(':')[1]

        if (chrom, pos) in snpDict:
            #print(chrom,pos,ref,alt,genotype,allele_depth)
            ref_depth = 0
            alt_dept = 0

            if genotype == '0/0' or genotype == '0':
                if ref == snpDict[(chrom, pos)][0] and alt == '.':
                    #print(chrom, pos, ref, alt, genotype, allele_depth)
                    ref_depth = int(allele_depth)
                else:
                    #print(chrom, pos, ref, alt, genotype, allele_depth)
                    continue

            elif genotype == '0/1' or genotype == '1/1' or genotype == '1':
                if ref != snpDict[(chrom, pos)][2]:
                    continue
                if alt == snpDict[(chrom, pos)][3]:
                    #print(chrom, pos, ref, alt, genotype, allele_depth)
                    ref_depth = int(allele_depth.split(',')[0])
                    alt_depth = int(allele_depth.split(',')[1])
                else:
                    ref_depth = int(allele_depth.split(',')[0]) + int(allele_depth.split(',')[1])
                    #print('Warning check ref and alt alleles: ', sample, chrom,pos,ref,alt,genotype,allele_depth, snpDict[(chrom, pos)][2], snpDict[(chrom, pos)][3])
                    continue

            elif genotype == '1/2':
                for depth in allele_depth.split(','):
                    ref_depth = ref_depth + int(depth)
                
                if sample == 'HER19xN8ma' and pos == 100538618:
                    ref_depth = 3
                    alt_depth = 8
                
                #print(sample, chrom, pos, ref, alt, genotype, allele_depth, snpDict[(chrom, pos)][2], snpDict[(chrom, pos)][3], ref_depth)

            else:
                #print('Warning check genotype format:', sample, chrom, pos, ref, alt, genotype, allele_depth, snpDict[(chrom, pos)][2], snpDict[(chrom, pos)][3])
                continue

            total_depth = ref_depth + alt_depth
            alt_prop = 'na'

            if total_depth > 0:
                alt_prop = alt_depth / total_depth            

            sampleDict[sample][(chrom, pos)] = [alt_depth, alt_prop]

            print(sample, chrom, pos, ref, alt, genotype, alt_depth, alt_prop)

    fi.close()

her19 Chr29 100538618 T C 1 260 0.9961685823754789
her19 Chr29 100538619 A C 1 235 0.9873949579831933
her19 Chr29 100538627 T G 1 260 0.9961685823754789
her19 Chr29 100538629 C G 1 260 0.9961685823754789
her19 Chr29 100538630 A C 1 260 0.9961685823754789
her19 Chr29 100538632 T C 1 260 0.9961685823754789
her19 Chr29 100538634 A G 1 260 0.9961685823754789
her19 Chr29 100538638 A T 1 260 0.9961685823754789
her19 Chr29 100538640 A G 1 260 0.9961685823754789
her19 Chr29 100538642 A T 1 260 0.9961685823754789
her19 Chr29 100538659 G A 1 224 0.9955555555555555
her19 Chr29 100538675 A C 1 197 0.985
her19 Chr29 100538679 C A 1 196 0.9849246231155779
her19 Chr29 100538688 C T 1 165 0.9821428571428571
her19 Chr29 100538693 T G 1 61 0.9384615384615385
her19 Chr29 100538702 C G 1 32 0.9411764705882353
her19 Chr29 100538703 C A 1 26 0.9285714285714286
her19 Chr29 100538705 T A 1 24 0.9230769230769231
her19 Chr29 100538709 C A 1 18 0.9
her19 Chr29 101567637 G T 1 61 0.8243243243243243
her19 Chr29 10

In [67]:
fo = open(outfile, 'w')

fo.write('chrom\tlocation\tref\talt\tgene\tconsequence\tamino_acids\tcodons\timpact')
for sample in sampleDict:
    fo.write('\t' + sample)
fo.write('\n')

for snp in snpDict:
    chrom,loc,ref,alt,gene,effect,aa,codon,impact = snpDict[snp]
    fo.write(chrom + '\t' + str(loc) + '\t' + ref + '\t' + alt + '\t' + gene + '\t' + effect + '\t' + aa + '\t' + codon + '\t' + impact)
    
    for sample in sampleDict:
        alt_prop = 0
        if snp in sampleDict[sample]:
            alt_prop = sampleDict[sample][snp][1]

        
        fo.write('\t' + str(alt_prop))
    fo.write('\n')
            
fo.close()